# Process the v43.4 Data into hdf5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import socket as socket
import os as os
import sys as sys
import multiprocessing as mp
import h5py
import allel

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/hapBLOCK/"  # The Path on Harvard Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

sys.path.insert(0, "/n/groups/reich/hringbauer/git/HAPSBURG/package/")  # hack to get local package first in path
from hapsburg.PackagesSupport.h5_python.h5_functions import merge_in_ld_map

compute-a-16-161.o2.rc.hms.harvard.edu
HSM O2 Computational partition detected.
/n/groups/reich/hringbauer/git/hapBLOCK
CPU Count: 32


In [53]:
path_ali_vcf = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf"
path_ali_stats = "/n/groups/reich/ali/chromosome_abnormality/coverage_stats_v43.4.tsv"
### Read Ali's coverage stats
#df = pd.read_csv("/n/groups/reich/ali/chromosome_abnormality/coverage_stats_v43.4.tsv", sep="\t")

### Downsample to 1240K data

In [2]:
def save_1240kmarkers(snp1240k_path="", marker_path="", ch=0):
    """Save all 1240 Markers of .snp eigenstrat file.
    to marker_path.
    ch: Chromosome. If null filter all of them"""
    df_snp = pd.read_csv(snp1240k_path, header=None, sep=r"\s+", engine="python")
    df_snp.columns = ["SNP", "chr", "map", "pos", "ref", "alt"]
    if ch>0:
        df_snp = df_snp[df_snp["chr"] == ch]
    print(f"Loaded {len(df_snp)} Chr.{ch} SNPs.")

    df_save = df_snp[["chr", "pos"]]
    df_save.to_csv(marker_path, sep="\t", header=None, index=False)
    print(f"Saved {len(df_save)} 1240k Markers on Chr. {ch} to {marker_path}")
    
def save_1240_1000g_kmarkers(ch=3, snp_path="", marker_path=""):
    """Save all 1240 and 1000G Markers of .snp eigenstrat file.
    to marker_path. Loads Ali Path file
    snp_path: Where to find the SNPs plus their types"""
    df1=pd.read_csv(snp_path, sep="\t", header=None)
    df1.columns=["chr", "pos", "pos1", "ref", "alt", "type"]
    print(f"Loaded {len(df1)} SNPs on Chr. {ch}")
    df2 = df1[df1["type"]=="1kg+1240k"]
    print(f"Loaded {len(df2)} Chr.{ch} SNPs in both 1240K and 1000G.")
    df_save = df2[["chr", "pos1"]]
    df_save.to_csv(marker_path, sep="\t", header=None, index=False)
    print(f"Saved {len(df_save)} 1240k+1000G Markers on Chr. {ch} to {marker_path}")
    
def bctools_filter_vcf(in_vcf_path="", out_vcf_path="", marker_path=""):
    """Same as PLINK, but with bcftools and directly via Marker Positions.
    filter_iids: Whether to use the .csv with Indivdiduals to extract"""
    !bcftools view -Oz -o $out_vcf_path -T $marker_path -M2 -v snps $in_vcf_path
    print("Finished BCF tools filtering.")
    
def bctools_filter_vcf_allvariants(in_vcf_path="", out_vcf_path="", marker_path=""):
    """Same as PLINK, but with bcftools and directly via Marker Positions.
    filter_iids: Whether to use the .csv with Indivdiduals to extract"""
    !bcftools view -Oz -o $out_vcf_path -T $marker_path -v snps $in_vcf_path
    print("Finished BCF tools filtering.")
    
def merge_vcfs(in_vcf_paths=[], out_vcf_path=""):
    """Merges Set of VCFs into one VCF. 
    in_vcf_paths: List of VCFs to merge
    out_vcf_path: Output of VCF"""
    paths_merge = " ".join(in_vcf_paths)
    !bcftools concat -n -o $out_vcf_path $paths_merge
    print("Finished BCF tools filtering.")
    
##############################################################
### Function Identical to vcf_to_hdf5.py in cluster/ folder

def vcf_to_1240K_hdf(in_vcf_path = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf",
                     path_vcf = "./data/vcf/1240k_v43/ch3.vcf.gz",
                     path_h5 = "./data/hdf5/1240k_v43/ch3.h5",
                     marker_path="./data/filters/ho_snps_bcftools_ch3.csv",
                     map_path="/n/groups/reich/DAVID/V43/V43.5/v43.5.snp",
                     ch=3):
    """Convert Ali's vcf to 1240K hdf5. 
    If marker_path empty, no SNP filtering done.
    If map_path empty, no genetic map is merged in.
    """ 
    if len(marker_path)>0:
        bctools_filter_vcf(in_vcf_path = in_vcf_path,
                           out_vcf_path= path_vcf,
                           marker_path = marker_path)
    print("Finished downsampling to 1240K")
    
    allel.vcf_to_hdf5(input=path_vcf, output=path_h5, 
                  fields = ['variants/*', 'calldata/*', "samples"], 
                      types={'samples': 'S32'},
                      compression="gzip") # Do the conversion to hdf5. Takes hours
    print("Finished conversion to hdf5!")
    
    if len(map_path)>0:
        merge_in_ld_map(path_h5=path_h5, 
                    path_snp1240k=map_path,
                    chs=[ch])

### Prepare HO and 1240K and 1240K 1000 G Markers [Finished]
Takes about 3 min for all autosomes

In [ ]:
%%time
### Same but for HO SNPs
for ch in range(1,23):
    save_1240kmarkers(snp1240k_path="/n/groups/reich/DAVID/V43/V43.5/v43.5_HO.snp",
                      marker_path=f"./data/filters/ho_snps_bcftools_ch{ch}.csv",
                      ch=ch)

In [ ]:
%%time
### Same but for HO SNPs
for ch in range(1,23):
    save_1240kmarkers(snp1240k_path="/n/groups/reich/DAVID/V43/V43.5/v43.5.snp",
                      marker_path=f"./data/filters/1240K_snps_bcftools_ch{ch}.csv",
                      ch=ch)

In [ ]:
%%time
### Save only SNPs in 1240K and 1000G
for ch in range(1,23):
    save_1240_1000g_kmarkers(snp_path=f"/n/groups/reich/datasets/1kg/snps_1kg_1240k/chr{ch}.bed",
                             marker_path=f"./data/filters/1240K_1000G/snps_bcftools_ch{ch}.csv",
                             ch=ch)

## Run through one chromosome: vcf -> 1240K vcf -> hdf5
Mainly to test timing, and prepare what is about to follow

## To 1240K VCF

In [42]:
%%time
bctools_filter_vcf(in_vcf_path = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf",
                   out_vcf_path= "./data/vcf/1240k_v43/ch3.vcf.gz",
                   marker_path = "./data/filters/1240K_snps_bcftools_ch3.csv")

^C
Finished BCF tools filtering.
CPU times: user 1.6 s, sys: 344 ms, total: 1.94 s
Wall time: 1min


In [8]:
print(f"Run finished")

Run finished


### Convert VCF to HDF5
Takes about 15min

In [ ]:
%%time
path_vcf = "./data/vcf/1240k_v43/ch1.vcf.gz"
path_h5 = "./data/hdf5/1240k_v43/ch1.h5"

allel.vcf_to_hdf5(input=path_vcf, output=path_h5, 
                  fields = ['variants/*', 'calldata/*', "samples"], compression="gzip") # Do the conversion to hdf5. Takes hours
print("Finished!")

In [1]:
print("Finished transformation to hdf5")

Finished transformation to hdf5


### Merge in genetic map

In [ ]:
%time
merge_in_ld_map(path_h5="./data/hdf5/1240k_v43/ch1.h5", 
                path_snp1240k="/n/groups/reich/DAVID/V43/V43.5/v43.5.snp",
                chs=[1])

## To 1240K VCF with all variants
vcf filterting takes about 2h for long chromosome

Transformation to hdf5 takes about 15 min

In [ ]:
%%time
bctools_filter_vcf_allvariants(in_vcf_path = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf",
                               out_vcf_path= "./data/vcf/1240k_v43_allsnps/ch3.vcf.gz",
                               marker_path = "./data/filters/ho_snps_bcftools_ch3.csv")

In [16]:
%%time
path_vcf = "./data/vcf/1240k_v43_allsnps/ch3.vcf.gz"
path_h5 = "./data/hdf5/1240k_v43_allsnps/ch3.h5"

allel.vcf_to_hdf5(input=path_vcf, output=path_h5, 
                  fields = ['variants/*', 'calldata/*', "samples"], compression="gzip") # Do the conversion to hdf5. Takes hours
print("Finished!")

Finished!
CPU times: user 15min 53s, sys: 1min 16s, total: 17min 10s
Wall time: 17min 13s


# All steps of the transformation to transformation of whole Chromosomes bundled up
To loop over multiple chromosomes

In [ ]:
%%time
vcf_to_1240K_hdf(in_vcf_path = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr4.bcf",
                 path_vcf = "./data/vcf/1240k_v43/ch4.vcf.gz",
                 path_h5 = "./data/hdf5/1240k_v43/ch4.h5",
                 marker_path="./data/filters/ho_snps_bcftools_ch4.csv",
                 map_path="/n/groups/reich/DAVID/V43/V43.5/v43.5.snp",
                 ch=4)
print(f"Finished running one chromosome")

# Bonus Task: Merge all vcfs into master vcf and create one master hdf5
Takes about ~5 min

In [5]:
%%time
base_folder_vcf = "./data/vcf/1240k_v43/ch"
out_vcf_path = "./data/vcf/1240k_v43/all_ch.vcf.gz"
paths_vcf = [base_folder_vcf + str(ch) + ".vcf.gz" for ch in range(1,23)]

merge_vcfs(in_vcf_paths=paths_vcf, out_vcf_path=out_vcf_path)

Checking the headers of 22 files.
Done, the headers are compatible.
Concatenating ./data/vcf/1240k_v43/ch1.vcf.gz	30.639150 seconds
Concatenating ./data/vcf/1240k_v43/ch2.vcf.gz	45.643698 seconds
Concatenating ./data/vcf/1240k_v43/ch3.vcf.gz	36.244412 seconds
Concatenating ./data/vcf/1240k_v43/ch4.vcf.gz	24.110753 seconds
Concatenating ./data/vcf/1240k_v43/ch5.vcf.gz	15.251328 seconds
Concatenating ./data/vcf/1240k_v43/ch6.vcf.gz	38.759324 seconds
Concatenating ./data/vcf/1240k_v43/ch7.vcf.gz	22.824363 seconds
Concatenating ./data/vcf/1240k_v43/ch8.vcf.gz	29.042413 seconds
Concatenating ./data/vcf/1240k_v43/ch9.vcf.gz	14.210376 seconds
Concatenating ./data/vcf/1240k_v43/ch10.vcf.gz	13.008295 seconds
Concatenating ./data/vcf/1240k_v43/ch11.vcf.gz	26.994599 seconds
Concatenating ./data/vcf/1240k_v43/ch12.vcf.gz	25.384168 seconds
Concatenating ./data/vcf/1240k_v43/ch13.vcf.gz	15.094192 seconds
Concatenating ./data/vcf/1240k_v43/ch14.vcf.gz	9.998397 seconds
Concatenating ./data/vcf/1240k_v

### And now transform the whole data to hdf5

In [12]:
%%time
out_path_h5="./data/hdf5/1240k_v43/all_ch.h5"
allel.vcf_to_hdf5(input=out_vcf_path, output=out_path_h5, chunk_length=10000, chunk_width=8,
                  fields = ['variants/*', 'calldata/*', "samples"], compression="gzip") # Do the conversion to hdf5. Takes 7h30

CPU times: user 6h 59min 46s, sys: 23min, total: 7h 22min 47s
Wall time: 7h 24min 39s


# Bonus: Create Variant only VCF

In [15]:
### index vcf file  -t
vcf_all = "./data/vcf/1240k_v43/all_ch.vcf.gz"
!bcftools index -f $vcf_all

In [16]:
print("Finished")

Finished


In [17]:
vcf_all = "./data/vcf/1240k_v43/all_ch.vcf.gz"
vcf_var_only = "./data/vcf/1240k_v43/1240k_vars.vcf.gz"

In [18]:
!bcftools view -G -o $vcf_var_only $vcf_all 

# Area 51
Test code here.

### Test vcf

In [ ]:
### index vcf file  -t
test = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf"
!bcftools view $test | head -25

In [23]:
!bcftools query -f '%POS\n' $vcf_var_only | wc -l

1100313


### Test hdf5

In [10]:
ch=1
with h5py.File(f"./data/hdf5/1240k_v43/ch{ch}.h5", "r") as f: # Load for Sanity Check. See below!
#g = h5py.File("./data/hdf5/HO_v43/ch3.h5", "r")
    print(list(f["variants"]))
    print(np.shape(f["calldata/GT"]))

['AF', 'ALT', 'BUF', 'CHROM', 'FILTER_PASS', 'ID', 'INFO', 'MAP', 'POS', 'QUAL', 'RAF', 'REF', 'altlen', 'is_snp', 'numalt']
(89082, 14523, 2)


In [11]:
with h5py.File(f"./data/hdf5/HO_v43/ch{ch}.h5", "r") as f:
    print(np.shape(f["calldata/GT"]))

(47705, 14523, 2)


In [13]:
samples = pd.Series(f["samples"][:])
samples[samples.str.contains("MA89")]

12483    MA89
dtype: object

In [79]:
snps = range(30000,30200)
j = 12483
ads = f["calldata/AD"][snps, j, :2]
gts = f["calldata/GT"][snps, j, :]
gp = f["calldata/GP"][snps, j, :]
df = pd.DataFrame({"ref":ads[:,0], "alt":ads[:,1], "gt0":np.sum(gts, axis=1)})

In [7]:
f.close()

In [30]:
f =  h5py.File(f"./data/hdf5/1240k_v43/all_ch.h5", "r")

In [31]:
np.shape(f["calldata/GT"])

(1100313, 14523, 2)

In [20]:
pd.value_counts(f["variants/CHROM"])

2     94173
1     89082
3     77601
6     75816
5     69354
4     68720
8     61091
7     59836
10    58763
11    54796
12    53884
9     50629
13    38994
14    36280
15    34407
16    34375
18    33902
17    29292
20    29053
19    18441
21    16031
22    15793
Name: /variants/CHROM, dtype: int64

# Plot Allele Frequencies